In [ ]:
"""
Make sure the PDB is in the same directory as notebook
    Name should be the 4 letter code of the pdb
Find N, O, P, atoms in the molecule. 
   Find the connected atom for each
For each functional atom, find the closest functional atom on a nearby residue
    Measure all possible constraints between them
    Uses some logic to filter constraints
Find residues close to the residues close to ligand functional atoms
    Measure constraints and use same filtering
    
    
To Come:
Metal support
More customizability
    turn on or off filtering
    additional functional atoms
Ligand atom connections and name from a Rosetta Param file

Fixes needed:
Getting the url retrieve to work for PDBfile
Uniprot filtering for residues

"""

In [ ]:
# USER change this for your PROTEIN and LIGAND
PDBfile = '3CTL.pdb'
LigName = 'S6P'
Chain = 'A'

In [ ]:
import numpy as np
from Bio.PDB import *
import array as arr
import os
import urllib
import re
from Bio import AlignIO
from Bio import SeqIO
from Bio.SeqUtils import seq3
from Bio.SeqUtils import seq1

In [ ]:
"""
Code for angles
https://stackoverflow.com/questions/35176451/python-code-to-calculate-angle-between-three-point-using-their-3d-coordinates
"""

def CalcAngle(XYZ1,XYZ2,XYZ3):
    X1, Y1, Z1 = XYZ1
    X2, Y2, Z2 = XYZ2
    X3, Y3, Z3 = XYZ3
    
    a = np.array([X1, Y1, Z1])
    b = np.array([X2, Y2, Z2])
    c = np.array([X3, Y3, Z3])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

"""
Code for distances
"""

def CalcDistance(XYZ1,XYZ2):
    X1, Y1, Z1 = XYZ1
    X2, Y2, Z2 = XYZ2
    
    a = np.array([X1, Y1, Z1])
    b = np.array([X2, Y2, Z2])
    
    return np.linalg.norm(a-b)

"""
Code for dihedrals
https://stackoverflow.com/questions/12785836/how-to-calculate-cartesian-coordinates-from-dihedral-angle-in-python
"""
def CalcDihedral(XYZ1,XYZ2,XYZ3,XYZ4):
    X1, Y1, Z1 = XYZ1
    X2, Y2, Z2 = XYZ2
    X3, Y3, Z3 = XYZ3
    X4, Y4, Z4 = XYZ4
    
    a = np.array([X1, Y1, Z1])
    b = np.array([X2, Y2, Z2])
    c = np.array([X3, Y3, Z3])
    d = np.array([X4, Y4, Z4])

    ba = b - a
    cb = c - b
    dc = d - c
    
    v1 = ba/np.linalg.norm(ba)
    v2 = cb/np.linalg.norm(cb)
    v3 = dc/np.linalg.norm(dc)
    v1v2 = np.cross(v1,v2)
    v2v3 = np.cross(v2,v3)
    return np.rad2deg(np.arccos(np.dot(v1v2/np.linalg.norm(v1v2),v2v3.T/np.linalg.norm(v2v3))))
    
def GetFuncAtom(resname):
    funcatom = []
    secondatom = []
    thirdatom = []
    if resname == 'CYS':
        funcatom = ['SG']
        secondatom = ['CB']
        thirdatom = ['CA']
    elif resname == 'ASP':
        funcatom = ['OD1', 'OD2']  
        secondatom = ['CG','CG']
        thirdatom = ['CB','CB']
    elif resname == 'GLU':
        funcatom = ['OE1', 'OE2']
        secondatom = ['CD', 'CD']
        thirdatom = ['CG','CG']
    elif resname == 'HIS':
        funcatom = ['NE2', 'ND1']
        secondatom = ['CE1', 'CE1']
        thirdatom = ['ND1','NE2'] 
    elif resname == 'LYS':
        funcatom = ['NZ']
        secondatom = ['CE']
        thirdatom = ['CD']
    elif resname == 'MET':
        funcatom = ['SD']
        secondatom = ['CG']
        thirdatom = ['CB']
    elif resname == 'ASN':
        funcatom = ['OD1', 'ND2']  
        secondatom = ['CG', 'CG']
        thirdatom = ['CB','CB']
    elif resname == 'GLN':
        funcatom = ['OE1', 'NE2'] 
        secondatom = ['CD', 'CD']
        thirdatom = ['CG','CG']
    elif resname == 'ARG':
        funcatom = ['NH1', 'NH2']
        secondatom = ['CZ', 'CZ']
        thirdatom = ['NE', 'NE']
    elif resname == 'SER':
        funcatom = ['OG']
        secondatom = ['CB']
        thirdatom = ['CA']
    elif resname == 'THR':
        funcatom = ['OG1']  
        secondatom = ['CB']
        thirdatom = ['CA']
    elif resname == 'TYR':
        funcatom = ['OH']  
        secondatom = ['CZ']
        thirdatom = ['CE1']

    secondatom.extend(['CA','C'])
    thirdatom.extend(['C', 'CA'])
    funcatom.extend(['N','O'])
    return funcatom , secondatom, thirdatom

def GetXYZ(PDBfile, funcatom, resname, resnum, Chain):
    with open(PDBfile, 'r') as pdb:
        for line in pdb:
            if line.startswith('ATOM') and line[17:20] == resname and line[21] == Chain and line[12:16].strip() == funcatom and line[22:28].strip() == resnum:                
                x = float(line[31:39].strip())
                y = float(line[39:47].strip())
                z = float(line[47:55].strip())
                break
    return [x,y,z]    

def FindNeighbors(LigAtomXYZ, structure):
    parser = PDBParser()
    a = arr.array('d',LigAtomXYZ )
    atom_list = Selection.unfold_entities(structure, 'A')
    ns = NeighborSearch(atom_list)
    neighbors = ns.search(a,3.2,level='R')

    reslist = []
    for residue in neighbors:
        if residue.id[0].startswith('H') or residue.id[0].startswith('W'):
            pass
        else:
            reslist.append(residue.get_resname()+str(residue.id[1]))
    return reslist

def GetPDBRes(pdbname, uniprotID, residues, pdb):
    pdbnums = []
    uniqresidues = []
    [uniqresidues.append(x) for x in residues if x not in uniqresidues]
    for res in uniqresidues:
        data = urllib.request.urlopen('http://www.bioinf.org.uk/servers/pdbsws/query.cgi?plain=1&qtype=ac&id=%s&res=%s' % (uniprotID, res)).readlines()
        r = re.findall(r"PDBAA: \w+", str(data))
        pdbresid = r[0].split()[1]
        pdbresid = seq3(pdbresid)
        ###  RESID below could run into issues when multiple PDBs: need some way to make sure RESID is for correct pdbname
        n = re.findall(r"RESID: \d+", str(data))
        pdbresnum = n[0].split()[1] 
        pdbnums.append(pdbresnum)
## Below is for getting rosetta numbering, ie starting at 1        
#         rcsb_res = []
#         rcsb_temp = []
#         atomlines = str(pdb).split('\\n')
#         for line in atomlines:
#             if line[0:4] == 'ATOM' and line[21] == 'A':
#                 resnum = line[22:26]
#                 resnum = resnum.strip()
#                 rcsb_temp.append(resnum)
#                 [rcsb_res.append(x) for x in rcsb_temp if x not in rcsb_res]
#         rosresnum = str(rcsb_res.index(str(pdbresnum))+1)
#         pdbres = pdbresid+rosresnum
#         pdbnums.append(pdbres)
    return pdbnums
        
def FindCatResUniprot(PDBfile):
    pdbname = PDBfile[:-3]
    resnums = []
    pdb = open(PDBfile, 'r').read()
    m = re.search('UNP\ +(\w+)', str(pdb))
    uniprotID = m.group(1)
    data = urllib.request.urlopen('https://www.uniprot.org/uniprot/%s.gff' % (uniprotID)).readlines()
    for line in data:
        line = str(line)
        new = line.split("\\t")
        try:
            new[2]
        except:
            pass
        else:
            if new[2] == 'Active site' or new[2] == 'Metal binding' or new[2] == 'Binding site':
                resnums.append(new[3])
    if not resnums:
        pass
    else:
        ResIDs = GetPDBRes(pdbname, uniprotID, resnums, pdb)
    return ResIDs

def FindSecondMolAtom(MoltoPDB, PDBtoMol, ligname, atomtofind, atomtoavoid=None):

    atomtofind = PDBtoMol[atomtofind]
    if atomtoavoid:
        atomtoavoid = PDBtoMol[atomtoavoid]
    molfile = urllib.request.urlopen('https://files.rcsb.org/ligands/view/%s_ideal.mol2' % (ligname)).read()
    lines = str(molfile).split('\\n')
    numtoatom = {}
    atomtonum = {}
    for line in lines:
        line = line.split()
        try:
            line[6]
        except:
            pass
        else:
            if not line[1].startswith('H'):
                numtoatom[line[0]] = line[1]
                atomtonum[line[1]] = line[0]
    for line in lines:
        line = line.split()
        try:
            line[3]
        except:
            pass
        else:
            if (line[1] == atomtonum[atomtofind] or line[2] == atomtonum[atomtofind]) and (line[1] in numtoatom) and (line[2] in numtoatom):
                if line[1] == atomtonum[atomtofind] and numtoatom[line[2]] != atomtoavoid:
                    secondmolatom = numtoatom[line[2]]
                    try:
                        FindThirdMolAtom(MoltoPDB, PDBtoMol, ligname, secondmolatom, atomtofind)
                    except:
                        pass
                    else:
                        thirdmolatom = FindThirdMolAtom(MoltoPDB, PDBtoMol, ligname, secondmolatom, atomtofind)
                        return MoltoPDB[secondmolatom] , thirdmolatom
                elif line[2] == atomtonum[atomtofind] and numtoatom[line[1]] != atomtoavoid:
                    secondmolatom = numtoatom[line[1]]
                    try:
                        FindThirdMolAtom(MoltoPDB, PDBtoMol, ligname, secondmolatom, atomtofind)
                    except:
                        pass
                    else:
                        thirdmolatom = FindThirdMolAtom(MoltoPDB, PDBtoMol, ligname, secondmolatom, atomtofind)
                        return MoltoPDB[secondmolatom] , thirdmolatom

def FindThirdMolAtom(MoltoPDB, PDBtoMol, ligname, atomtofind, atomtoavoid):
    molfile = urllib.request.urlopen('https://files.rcsb.org/ligands/view/%s_ideal.mol2' % (ligname)).read()
    lines = str(molfile).split('\\n')
    numtoatom = {}
    atomtonum = {}
    for line in lines:
        line = line.split()
        try:
            line[6]
        except:
            pass
        else:
            if not line[1].startswith('H'):
                numtoatom[line[0]] = line[1]
                atomtonum[line[1]] = line[0]

    for line in lines:
        line = line.split()
        try:
            line[3]
        except:
            pass
        else:
            if (line[1] == atomtonum[atomtofind] or line[2] == atomtonum[atomtofind]) and (line[1] in numtoatom) and (line[2] in numtoatom):
                if line[1] == atomtonum[atomtofind] and numtoatom[line[2]] != atomtoavoid:
                    secondmolatom = numtoatom[line[2]]
                elif line[2] == atomtonum[atomtofind] and numtoatom[line[1]] != atomtoavoid:
                    secondmolatom = numtoatom[line[1]]
    return MoltoPDB[secondmolatom]
    

def WriteCSTBlock(PDBname, LigName, numcst, Residue1, resnum, Chain1, Atoms1, Residue2, resnum2, Chain2, Atoms2, distAB, angleA, angleB, torsionA, torsionB, torsionAB):
    with open("%s_%s.enzdes.cst" % (LigName, PDBname), 'a') as f:
        f.write("""
CST::BEGIN
  TEMPLATE::   ATOM_MAP: 1 atom_name: %s
  TEMPLATE::   ATOM_MAP: 1 residue3: %s
  
  TEMPLATE::   ATOM_MAP: 2 atom_name: %s
  TEMPLATE::   ATOM_MAP: 2 residue3: %s
  
  CONSTRAINT:: distanceAB: %s 0.2 100.00 0
  CONSTRAINT:: angle_A: %s 10.0 50.0 360.0 
  CONSTRAINT:: angle_B: %s 10.0 50.0 360.0 
  CONSTRAINT:: torsion_A: %s 10.0 50.0 360.0 
  CONSTRAINT:: torsion_B: %s 10.0 50.0 360.0   
  CONSTRAINT:: torsion_AB: %s 10.0 50.0 360.0     
CST::END  
    """ % ( ' '.join([str(elem) for elem in Atoms1]), Residue1, ' '.join([str(elem) for elem in Atoms2]), Residue2, distAB, angleA, angleB, torsionA, torsionB, torsionAB))
    with open("%s_%s.enzdes.header" % (LigName, PDBname), 'a') as f:
        f.write("""REMARK 666 MATCH TEMPLATE %s %s %s MATCH MOTIF %s %s %s %s 1\n""" % (Chain1, Residue1, resnum, Chain2, Residue2, resnum2, numcst))

def WriteCSTBlockCheck(PDBname, LigName, numcst, Residue1, resnum, Chain1, Atoms1, Residue2, resnum2, Chain2, Atoms2, distAB, angleA, angleB, torsionA, torsionB, torsionAB):
    with open("%s_%s.enzdes.cst" % (LigName, PDBname), 'a') as f:
        f.write("""
CST::BEGIN
  TEMPLATE::   ATOM_MAP: 1 atom_name: %s
  TEMPLATE::   ATOM_MAP: 1 residue3: %s
  
  TEMPLATE::   ATOM_MAP: 2 atom_name: %s
  TEMPLATE::   ATOM_MAP: 2 residue3: %s
  
  CONSTRAINT:: distanceAB: %s 0.2 100.00 1 0\n""" % (' '.join([str(elem) for elem in Atoms1]), Residue1, ' '.join([str(elem) for elem in Atoms2]), Residue2, distAB))
        if angleA:
            f.write("  CONSTRAINT:: angle_A: %s 10.0 50.0 360.0\n" % (angleA))
        if angleB:
            f.write("  CONSTRAINT:: angle_B: %s 10.0 50.0 360.0\n" % (angleB))
        if torsionA:           
            f.write("  CONSTRAINT:: torsion_A: %s 10.0 50.0 360.0\n" % (torsionA))
        if torsionB:
            f.write("  CONSTRAINT:: torsion_B: %s 10.0 50.0 360.0\n" % (torsionB))
        if torsionAB:
            f.write("  CONSTRAINT:: torsion_AB: %s 10.0 50.0 360.0\n" % (torsionAB))    
        f.write("CST::END\n\n")  

    with open("%s_%s.enzdes.header" % (LigName, PDBname), 'a') as f:
        f.write("""REMARK 666 MATCH TEMPLATE %s %s %s MATCH MOTIF %s %s %s %s 1\n""" % (Chain1, Residue1, resnum, Chain2, Residue2, resnum2, numcst))

Angle = {'ALA': {'N': 120.0, 'O': 120.0},
         'CYS': {'N': 120.0, 'O': 120.0 , 'SG': 109.5},
         'ASP': {'N': 120.0, 'O': 120.0 , 'OD1': 120.0, 'OD2': 120.0},                 
         'GLU': {'N': 120.0, 'O': 120.0 , 'OE1': 120.0, 'OE2': 120.0},   
         'PHE': {'N': 120.0, 'O': 120.0 },
         'GLY': {'N': 120.0, 'O': 120.0 },
         'HIS': {'N': 120.0, 'O': 120.0 , 'ND1': 120.0, 'NE2': 120.0},
         'ILE': {'N': 120.0, 'O': 120.0 },
         'LYS': {'N': 120.0, 'O': 120.0 , 'NZ': 109.5},
         'LEU': {'N': 120.0, 'O': 120.0 },
         'MET': {'N': 120.0, 'O': 120.0 , 'SD': 109.5},
         'ASN': {'N': 120.0, 'O': 120.0 , 'OD1': 120.0, 'ND2': 109.5},
         'PRO': {'N': 120.0, 'O': 120.0 },
         'GLN': {'N': 120.0, 'O': 120.0 , 'OE1': 120.0, 'NE2': 109.5},
         'ARG': {'N': 120.0, 'O': 120.0 , 'NH1': 120.0, 'NH2': 120},
         'SER': {'N': 120.0, 'O': 120.0 , 'OG': 109.5},
         'THR': {'N': 120.0, 'O': 120.0 , 'OG1': 109.5},
         'VAL': {'N': 120.0, 'O': 120.0 },
         'TRP': {'N': 120.0, 'O': 120.0 },
         'TYR': {'N': 120.0, 'O': 120.0 , 'OH': 109.5},
                  }        

def CheckAngle(Residue, atom, angle):
    angle_ideal = Angle[Residue][atom]
    if angle > (angle_ideal - 20) and angle < (angle_ideal + 20):
        angle = angle_ideal
    else:
        angle = None
        
    return angle

def CheckTorsion(Residue, atom, torsion) :   
    if torsion < 20:
        torsion = 0.0
    elif torsion > 160:
        torsion = 180.0
    else:
        torsion = None
    
    return torsion

def CheckTetrahedral(angle):
    if angle < 129.5 and angle > 89.5:
        angle = 109.5
    else:
        angle = None
    return angle  

def MakeLigDicts(LigName):
    PDBtoMol = {}
    MoltoPDB = {}
    Molnumbering = []
    PDBnumbering = []
    molfile = urllib.request.urlopen('https://files.rcsb.org/ligands/view/%s_ideal.mol2' % (LigName)).read()
    lines = str(molfile).split('\\n')
    for line in lines:
        line = line.split()
        try:
            line[6]
        except:
            pass
        else:
            Molnumbering.append(line[1])

    pdbfile = urllib.request.urlopen('https://files.rcsb.org/ligands/view/%s_ideal.pdb' % (LigName)).read()
    lines = str(pdbfile).split('\\n')
    for line in lines:
        line = line.split()
        try:
            line[8]
        except:
            pass
        else:
            PDBnumbering.append(line[2])
    counter = 0
    for entry in Molnumbering:
        MoltoPDB[Molnumbering[counter]] = PDBnumbering[counter]
        PDBtoMol[PDBnumbering[counter]] = Molnumbering[counter]
        counter += 1
    return MoltoPDB , PDBtoMol


In [ ]:

LigAtomInterest = {}
LigAtomXYZ = {}


parser = PDBParser()
try:
    open(PDBfile)
except:
    urllib.request.urlretrieve('https://files.rcsb.org/download/'+PDBfile, PDBfile)

structures = parser.get_structure("1", PDBfile)
structure = structures[0]


with open(PDBfile, 'r') as pdb:
    for line in pdb:
        if line.startswith('HETATM') and line[17:20] == LigName and line[21] == Chain and (line[13] == 'O' or line[13] == 'N' or line[13] == 'P'):                
            LigAtomInterest[line[11:17].strip()] = [float(line[31:39].strip()), float(line[39:47].strip()), float(line[47:55].strip())]
        if line.startswith('HETATM') and line[17:20] == LigName and line[21] == Chain:
            LigAtomXYZ[line[11:17].strip()] = [float(line[31:39].strip()), float(line[39:47].strip()), float(line[47:55].strip())]
            
LigAtomNeighbors = {}
for atom in LigAtomInterest:
    LigAtomNeighbors[atom] = FindNeighbors(LigAtomInterest[atom], structure)
    
CatReslist = [str(x) for x in range(1,9999)]
#CatReslist = FindCatResUniprot(PDBfile)
MoltoPDB , PDBtoMol = MakeLigDicts(LigName)

In [ ]:
numcst = 1
ResidueList = []
for atom in LigAtomNeighbors:
    for residue in LigAtomNeighbors[atom]:
        resname = residue[0:3]
        resnum = residue[3:]
        if resnum in CatReslist:
            funcatoms , secondpdbatoms, thirdpdbatoms = GetFuncAtom(resname)
            counter = 0
            for funcatom in funcatoms:
                secondpdbatom = secondpdbatoms[counter]
                thirdpdbatom = thirdpdbatoms[counter]
                
                secondligatom , thirdligatom = FindSecondMolAtom(MoltoPDB, PDBtoMol, LigName, atom)
                
                funcxyz = GetXYZ(PDBfile, funcatom, resname, resnum, Chain)
                secondxyz = GetXYZ(PDBfile, secondpdbatom, resname, resnum, Chain)
                thirdxyz = GetXYZ(PDBfile, thirdpdbatom, resname, resnum, Chain)
                
                ligxyz = LigAtomInterest[atom]
                secondligxyz = LigAtomXYZ[secondligatom]
                thirdligxyz = LigAtomXYZ[thirdligatom]
                dist = CalcDistance(funcxyz,ligxyz)
                if dist < 3.0:
                    ResidueList.append(residue)
                    angleA = CalcAngle(secondxyz, funcxyz, ligxyz)
                    angleB = CalcAngle(secondligxyz, ligxyz, funcxyz)
                    torsionA = CalcDihedral(thirdxyz, secondxyz, funcxyz, ligxyz)
                    torsionB = CalcDihedral(funcxyz, ligxyz, secondligxyz, thirdligxyz) 
                    torsionAB = CalcDihedral(secondxyz, funcxyz, ligxyz, secondligxyz)
                    print(resname, resnum)
                    print(resname, funcatom)
                    angleA = CheckAngle(resname, funcatom, angleA)
                    angleB = CheckTetrahedral(angleB)
                    torsionA = CheckTorsion(resname, funcatom, torsionA)
                    torsionB = CheckTorsion(resname, funcatom, torsionB)
                    torsionAB = CheckTorsion(resname, funcatom, torsionAB)                    
                    WriteCSTBlockCheck(PDBfile[:-4], LigName, numcst, resname, resnum, Chain, [funcatom, secondpdbatom, thirdpdbatom], LigName, '1', 'X', [atom, secondligatom, thirdligatom], round(dist,1), angleA, angleB, torsionA, torsionB, torsionAB)
                    numcst += 1
                counter += 1

In [ ]:
ResList = []
[ResList.append(x) for x in ResidueList if x not in ResList]
for residueA in ResList:
    firstatoms , secondatoms , thirdatoms = GetFuncAtom(residueA[:3])
    counterA = 0
    for firstatom in firstatoms:
        secondatom = secondatoms[counterA]
        thirdatom = thirdatoms[counterA]
        
        neighborlist = FindNeighbors(GetXYZ(PDBfile, firstatom, residueA[:3], residueA[3:], Chain), structure)
        for residueB in neighborlist:
            if residueA == residueB:
                pass
            else:
                fourthatoms, fifthatoms, sixthatoms = GetFuncAtom(residueB[:3])
                counterB = 0
                for fourthatom in fourthatoms:
                    if (firstatom == 'N' or firstatom == 'O') or (fourthatom == 'N' or fourthatom == 'O'): # filters out bb to bb if 'and', filters any bb if or
                        counterB += 1
                    else:
                        fifthatom = fifthatoms[counterB]
                        sixthatom = sixthatoms[counterB]

                        firstxyz = GetXYZ(PDBfile, firstatom, residueA[:3], residueA[3:], Chain)
                        secondxyz = GetXYZ(PDBfile, secondatom, residueA[:3], residueA[3:], Chain)                
                        thirdxyz = GetXYZ(PDBfile, thirdatom, residueA[:3], residueA[3:], Chain)                
                        fourthxyz = GetXYZ(PDBfile, fourthatom, residueB[:3], residueB[3:], Chain)
                        fifthxyz = GetXYZ(PDBfile, fifthatom, residueB[:3], residueB[3:], Chain)
                        sixthxyz = GetXYZ(PDBfile, sixthatom, residueB[:3], residueB[3:], Chain)
                        dist = CalcDistance(firstxyz,fourthxyz)
                        if dist < 3.0:
                            angleA = CalcAngle(secondxyz, firstxyz, fourthxyz)
                            angleB = CalcAngle(fifthxyz, fourthxyz, firstxyz)
                            torsionA = CalcDihedral(thirdxyz, secondxyz, firstxyz, fourthxyz)
                            torsionB = CalcDihedral(firstxyz, fourthxyz, fifthxyz, sixthxyz) 
                            torsionAB = CalcDihedral(secondxyz, firstxyz, fourthxyz, fifthxyz)
                            print(residueA, residueB)
                            angleA = CheckAngle(residueA[:3], firstatom, angleA)
                            angleB = CheckAngle(residueB[:3], fourthatom, angleB)
                            torsionA = CheckTorsion(residueA[:3], firstatom, torsionA)
                            torsionB = CheckTorsion(residueB[:3], fourthatom, torsionB)
                            torsionAB = CheckTorsion(residueA[:3], firstatom, torsionAB)
                            WriteCSTBlockCheck(PDBfile[:-4], LigName, numcst, residueA[:3], residueA[3:], Chain, [firstatom, secondatom, thirdatom], residueB[:3], residueB[3:], Chain, [fourthatom, fifthatom, sixthatom], round(dist,1), angleA, angleB, torsionA, torsionB, torsionAB)
                            numcst += 1  
                        counterB += 1
                
                
        counterA += 1